<a href="https://colab.research.google.com/github/Junha9/AI/blob/master/Self_Learning_ML_DL/Chapter_07_2_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# import statements
from torchvision.datasets import FashionMNIST
from sklearn.model_selection import train_test_split

In [3]:
# download the FashionMNIST data
fm_train = FashionMNIST(root='.', train=True, download=True)
fm_test = FashionMNIST(root='.', train=False, download=True)

In [4]:
# Check the dataset. It is composed of torch.Tensor class.
print(type(fm_train.data))
print(fm_train.data.shape, fm_test.data.shape)
print(fm_train.targets.shape, fm_test.targets.shape)

<class 'torch.Tensor'>
torch.Size([60000, 28, 28]) torch.Size([10000, 28, 28])
torch.Size([60000]) torch.Size([10000])


In [5]:
# pytorch tensor also supports broadcasting just like numpy array.
train_input = fm_train.data
train_target = fm_train.targets
train_scaled = train_input / 255.0

In [7]:
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)
print(train_scaled.shape, val_scaled.shape)

torch.Size([48000, 28, 28]) torch.Size([12000, 28, 28])
